In [ ]:
import settings
import geometry_utils
import hypercolumn
import sys
from datetime import datetime as dt
import numpy as np
import sklearn.ensemble
import sklearn.metrics
from joblib import dump, load
import io
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
import scipy.misc
from keras import backend as K
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import cv2
from numpy.random import random as rnd
from skimage.transform import resize
import os
from PIL import Image
from poisson_disk import PoissonDiskSampler
import skimage.morphology
import skimage.measure
import scipy.stats
import dataset_utils
import joblib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def main():
    cub = dataset_utils.CUB_200_2011(settings.CUB_FOLDER_PATH)
    imgs_addr = cub.image_addrs()
    trimaps_addr = cub.trimap_addrs()
    imgs_anno = dataset_utils.BerkeleyAnnotaionHelper(cub)
    IDtrain, IDtest = cub.train_test_id()
    
    dh = hypercolumn.hypercolumn()
    
    part_name= 'head'
    
    pos_x = []
    neg_x = []
    
    
    
    index=0
    start = dt.now()
    
    d=0
    for i, img_id in enumerate(IDtrain):
        
        index +=1
        print ("done" ,index, "in", (dt.now() - start))
        img = cv2.imread(imgs_addr[img_id])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img_gray = cv2.imread(trimaps_addr[img_id], cv2.IMREAD_GRAYSCALE)
        
        height,width,domain = img.shape
        print(imgs_addr[img_id])
        
        part_box = imgs_anno.annotation(img_id, part_name)
        if not part_box.is_valid():
            continue
        pos, neg = dh.image_point_features(img,img_gray, part_box, part_name)
         
        pos_x.append(pos)
        neg_x.append(neg)
        
        
    pos_x = np.vstack(pos_x)
    neg_x = np.vstack(neg_x)
    
    pos_y = np.ones(np.array(pos_x).shape[0], dtype=np.int)
    
    neg_y = np.zeros(np.array(neg_x).shape[0], dtype=np.int)
    
    X = np.vstack((pos_x, neg_x))
    y = np.concatenate((pos_y, neg_y))
    
    print ('started learning')
    l_start = dt.now() 
    #100
    model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, max_depth=20, n_jobs=1, random_state=None)
    model.fit(X, y)
    
    preds = model.predict_proba(X)
    print (sklearn.metrics.auc(y, preds[:, 0]))
    
    joblib.dump(model, '%s_model(Vgg19-tree100).mdl' % part_name, compress=3)
    
    #150
    model = sklearn.ensemble.RandomForestClassifier(n_estimators=150, max_depth=20, n_jobs=1, random_state=None)
    model.fit(X, y)
    
    preds = model.predict_proba(X)
    print (sklearn.metrics.auc(y, preds[:, 0]))
    
    joblib.dump(model, '%s_model(Vgg19-tree150).mdl' % part_name, compress=3)
    
    #200
    model = sklearn.ensemble.RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=1, random_state=None)
    model.fit(X, y)
    
    preds = model.predict_proba(X)
    print (sklearn.metrics.auc(y, preds[:, 0]))
    
    joblib.dump(model, '%s_model(Vgg19-tree200).mdl' % part_name, compress=3)
    
    print ('learned in', (dt.now() - l_start))
    print ('finsihed in', (dt.now() - start))
    
if __name__ == '__main__':
    main()